In [1]:
# default_exp python_function_code

In [2]:
# export
import pandas as pd
import numpy as np
import ast
import astunparse

import multiprocessing.pool
import tqdm

from github_search import python_tokens

In [3]:
%cd ..

/home/kuba/Projects/github_search


In [4]:
%%time
python_files_df = pd.read_feather("data/all_crawled_python_files.feather")

CPU times: user 11.1 s, sys: 8.08 s, total: 19.2 s
Wall time: 19 s


In [5]:
paperswithcode_with_tasks_df = pd.read_csv("data/paperswithcode_with_tasks.csv") 

In [6]:
python_files_df.head()

,content,path,repo_name
0,#!/usr/bin/env python3\nimport subprocess\nimp...,run-tests.py,trangvu/ape-npi
1,#!/usr/bin/env python3\n# Author: Rico Sennric...,scripts/apply_bpe.py,trangvu/ape-npi
2,#!/usr/bin/env python3\n\nimport argparse\npar...,scripts/concat-bpe.py,trangvu/ape-npi
3,#!/usr/bin/env python3\nimport argparse\nimpor...,scripts/copy-model.py,trangvu/ape-npi
4,#!/usr/bin/env python3\n\nimport argparse\nfro...,scripts/coverage.py,trangvu/ape-npi


In [7]:
selected_repos_python_files_df = python_files_df[python_files_df['repo_name'].isin(paperswithcode_with_tasks_df['repo'].iloc[:100])]

In [8]:
selected_repos_python_files_df.shape

(5464, 3)

In [9]:
def get_per_repo_similarities(
    paperswithcode_df: pd.DataFrame,
    repo_grouped_contents: dict,
    similar_col: str,
    bow_vectorizer_class: str,
):
    bow_vectorizer = getattr(feature_extraction.text, bow_vectorizer_class)()

    bow_vectorizer.fit(paperswithcode_df[similar_col])
    return {
        repo: metrics.pairwise.cosine_similarity(
            bow_vectorizer.transform([abstract]),
            bow_vectorizer.transform(repo_grouped_contents[repo]),
        )[0]
        for (repo, abstract) in paperswithcode_df[["repo", similar_col]].itertuples(
            index=False
        )
        if len(repo_grouped_contents[repo]) > 0
    }



In [10]:
repo_grouped_contents = {
    repo: python_files_df[python_files_df["repo_name"] == repo]["content"]
    for repo in tqdm.tqdm(paperswithcode_with_tasks_df["repo"].iloc[:100])
}

100%|██████████| 100/100 [00:09<00:00, 10.53it/s]


In [11]:
%%time

d = dict(tuple(selected_repos_python_files_df.groupby("repo_name")[['content']]))
for (k, v) in d.items():
    d[k] = v.values[0]

CPU times: user 3.6 ms, sys: 0 ns, total: 3.6 ms
Wall time: 3.08 ms


In [12]:
type(d[k][0])

str

In [13]:
d[k][0]

'import io\nimport os\nfrom os import path\nimport re\nfrom setuptools import setup, find_packages\n# To use consisten encodings\nfrom codecs import open\n\n# Function from: https://github.com/pytorch/vision/blob/master/setup.py\n\n\ndef read(*names, **kwargs):\n    with io.open(\n        os.path.join(os.path.dirname(__file__), *names),\n        encoding=kwargs.get("encoding", "utf8")\n    ) as fp:\n        return fp.read()\n\n# Function from: https://github.com/pytorch/vision/blob/master/setup.py\n\n\ndef find_version(*file_paths):\n    version_file = read(*file_paths)\n    version_match = re.search(r"^__version__ = [\'\\"]([^\'\\"]*)[\'\\"]",\n                              version_file, re.M)\n    if version_match:\n        return version_match.group(1)\n    raise RuntimeError("Unable to find version string.")\n\nhere = path.abspath(path.dirname(__file__))\n\n# Get the long description from the README file\nwith open(path.join(here, \'README.md\'), encoding=\'utf-8\') as readme_file:

In [14]:
example_file = python_files_df["content"].iloc[1]

In [15]:
python_files_df["content"].iloc[6]

"#!/usr/bin/env python3\n\nimport itertools\nimport argparse\nimport re\nimport os\nimport sys\n\nparser = argparse.ArgumentParser()\nparser.add_argument('log_files', nargs='+')\nparser.add_argument('--dev-prefix')\nparser.add_argument('--score', choices=('ter', 'bleu', 'wer'))\nparser.add_argument('--task-name')\nparser.add_argument('--time', action='store_true')\nparser.add_argument('--params', action='store_true')\n\nparser.add_argument('--ter', action='store_true')\nparser.add_argument('--bleu', action='store_true')\nparser.add_argument('--wer', action='store_true')\nparser.add_argument('--cer', action='store_true')\nparser.add_argument('--bleu1', action='store_true')\nparser.add_argument('--loss', action='store_true')\n\n\ndef print_scores(log_file, time=False, label=None):\n    with open(log_file) as log_file:\n        scores = {}\n        times = {}\n        current_step = 0\n        max_step = 0\n        starting_time = None\n        param_count = None\n\n        def read_time(

In [16]:
python_files_df.shape

(3169680, 3)

In [17]:
file_ast = ast.parse(example_file)  # .body[3]

In [18]:
# export


def select_class_names(lines):
    return [
        line.strip()
        for line in lines
        if line.lstrip().startswith("class ") and line.rstrip().endswith(":")
    ]


def select_function_names(lines):
    return [line.strip() for line in lines if line.lstrip().startswith("def ")]


def select_lines(text, use_function_names=True, use_class_names=False):
    lines = text.split("\n")
    selected_lines = []
    if use_class_names:
        selected_lines = selected_lines + select_class_names(lines)
    if use_function_names:
        selected_lines = selected_lines + select_function_names(lines)

    return selected_lines


def select_functions(file_contents):
    return [
        elem
        for elem in ast.parse(file_contents).body
        if elem.__class__ is ast.FunctionDef
    ]

In [19]:
fn_def = ast.parse("def foo(x): return x + 1").body[0]

In [20]:
print(astunparse.unparse(fn_def).strip())

def foo(x):
    return (x + 1)


In [21]:
example_file = python_files_df["content"].iloc[8]
fn_defs = select_functions(example_file)

In [22]:
fn_defs

In [23]:
astunparse.unparse(fn_defs[0])

"\n\ndef create_parser():\n    parser = argparse.ArgumentParser(formatter_class=argparse.RawDescriptionHelpFormatter, description='learn BPE-based word segmentation')\n    parser.add_argument('--input', '-i', type=argparse.FileType('r'), default=sys.stdin, metavar='PATH', help='Input text (default: standard input).')\n    parser.add_argument('--output', '-o', type=argparse.FileType('w'), default=sys.stdout, metavar='PATH', help='Output file for BPE codes (default: standard output)')\n    parser.add_argument('--symbols', '-s', type=int, default=10000, help='Create this many new symbols (each representing a character n-gram) (default: %(default)s))')\n    parser.add_argument('--verbose', '-v', action='store_true', help='verbose mode.')\n    parser.add_argument('--min-freq', '-f', type=int, default=2, help='minimum word frequency')\n    return parser\n"

In [24]:
astunparse.unparse(fn_defs[-1])

"\n\ndef prune_stats(stats, big_stats, threshold):\n    'Prune statistics dict for efficiency of max()\\n\\n    The frequency of a symbol pair never increases, so pruning is generally safe\\n    (until we the most frequent pair is less frequent than a pair we previously pruned)\\n    big_stats keeps full statistics for when we need to access pruned items\\n    '\n    for (item, freq) in list(stats.items()):\n        if (freq < threshold):\n            del stats[item]\n            if (freq < 0):\n                big_stats[item] += freq\n            else:\n                big_stats[item] = freq\n"

In [25]:
import astunparse

astunparse.unparse(fn_defs[-1]).strip()

"def prune_stats(stats, big_stats, threshold):\n    'Prune statistics dict for efficiency of max()\\n\\n    The frequency of a symbol pair never increases, so pruning is generally safe\\n    (until we the most frequent pair is less frequent than a pair we previously pruned)\\n    big_stats keeps full statistics for when we need to access pruned items\\n    '\n    for (item, freq) in list(stats.items()):\n        if (freq < threshold):\n            del stats[item]\n            if (freq < 0):\n                big_stats[item] += freq\n            else:\n                big_stats[item] = freq"

In [26]:
# export
import io
import tokenize
import keyword
import re


PYTHON_KEYWORDS = set(keyword.kwlist)


def tokenize_snakecase(identifier):
    return identifier.split("_")


def tokenize_camelcase(identifier):
    matches = re.finditer(
        ".+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)", identifier
    )
    return [m.group(0) for m in matches]


def tokenize_python(identifier, lowercase=False):
    if "_" in identifier:
        tokens = tokenize_snakecase(identifier)
    else:
        tokens = tokenize_camelcase(identifier)
    return [t.lower() for t in tokens]


def get_file_variable_token_set(file_text, min_token_length=2, lowercase=True):
    token_infos = list(tokenize.generate_tokens(io.StringIO(file_text).readline))
    raw_tokens = [t.string for t in token_infos if t.type == 1]
    all_tokens = (tokenize_python(t, lowercase) for t in raw_tokens)
    all_tokens = [
        token
        for tokens in all_tokens
        for token in tokens
        if len(token) > min_token_length and not token in PYTHON_KEYWORDS
    ]
    return set(all_tokens)


def maybe_get_file_variable_token_string(file_text, min_token_length=2):
    try:
        tokens = get_file_variable_token_set(file_text)
    except:
        return None
    return " ".join(tokens)

In [27]:
python_files_df.iloc[:10000]["repo_name"].value_counts()

tensorflow/models                                8094
huggingface/transformers                         1242
vincentzhang/faster-rcnn-fcn                       90
mobvoi/wenet                                       83
cfedermann/Appraise                                59
trangvu/ape-npi                                    50
testingautomated-usi/selforacle                    43
repo_name                                          35
bfelbo/deepmoji                                    34
claudiogreco/coling18-gte                          27
facebookresearch/fastText                          24
ruotianluo/DiscCaptioning                          23
SJTU-lqiu/QA4IE                                    21
kchengiva/DecoupleGCN-DropGraph                    20
mravanelli/theano-kaldi-rnn                        16
harryhan618/LaneNet                                16
xiezheng-cs/DTQ                                    16
feifengwhu/question_attention                      16
yanggeng1995/GAN-TTS        

In [28]:
example_selected_lines = select_lines(
    " ".join(
        python_files_df[python_files_df["repo_name"] == "transformers"][
            "content"
        ].dropna()
    )
)

In [29]:
# export


def get_functions(code_ast, depth):
    if depth == 0:
        return None
    if type(code_ast) is ast.FunctionDef:
        return (code_ast.name, astunparse.unparse(code_ast).strip())
    elif type(code_ast) is ast.ClassDef:
        return (code_ast.name, astunparse.unparse(code_ast).strip())
    elif hasattr(code_ast, "body"):
        maybe_functions = [get_functions(item, depth) for item in code_ast.body]
        return [fn for fn in maybe_functions if not fn is None]
    else:
        return None


def get_function_tuples(code, max_depth):
    try:
        code_ast = ast.parse(code)
        return [
            fn_data for fn_data in get_functions(code_ast, max_depth) if type(fn_data) is tuple
        ]
    except (SyntaxError, multiprocessing.pool.MaybeEncodingError, RecursionError) as e:
        return []

In [30]:
def get_file_function_df(filename, file_contents):
    return pd.DataFrame(
        [(filename,) + tp for tp in get_function_tuples(file_contents)],
        columns=["file_path", "function_name", "function_code"],
    )

In [31]:
python_files_df.columns

Index(['content', 'path', 'repo_name'], dtype='object')

In [32]:
# export
import tqdm

def get_function_data_df(files_df, max_depth=10):
    tqdm.auto.tqdm.pandas(desc="making python functions df")
    return pd.DataFrame(
        list(
            files_df.progress_apply(
                lambda row: [
                    (row["repo_name"], row["path"]) + tp
                    for tp in get_function_tuples(row["content"], max_depth)
                ],
                axis=1,
            )
            .explode()
            .dropna()
            .values
        ),
        columns=["repo_name", "path", "function_name", "function_code"],
    )

In [33]:
list(selected_repos_python_files_df.iloc[:100].itertuples(index=False))

[Pandas(content='import os\nimport re\nimport setuptools\nfrom pathlib import Path\n\np = Path(__file__)\n\nsetup_requires = [\n    \'numpy\',\n    \'pytest-runner\'\n]\n\ninstall_requires = [\n]\ntest_require = [\n    \'pytest-cov\',\n    \'pytest-html\',\n    \'pytest\'\n]\n\nsetuptools.setup(\n    name="tfdbonas",\n    version=\'0.1.0\',\n    python_requires=\'>3.5\',\n    author="Koji Ono",\n    author_email="kbu94982@gmail.com",\n    description="Tensorflow Deep Bayes Optimization for Neural Network Architecture Search (tfDBONAS)",\n    url=\'https://github.com/0h-n0/tfdbonas\',\n    long_description=(p.parent / \'README.md\').open(encoding=\'utf-8\').read(),\n    long_description_content_type=\'text/markdown\',\n    packages=setuptools.find_packages(),\n    install_requires=install_requires,\n    setup_requires=setup_requires,\n    tests_require=test_require,\n    extras_require={\n        \'docs\': [\n            \'sphinx >= 1.4\',\n            \'sphinx_rtd_theme\']},\n    class

In [35]:
%%time
get_function_data_df(python_files_df)

making python functions df:   0%|          | 0/3169680 [00:00<?, ?it/s]

CPU times: user 2h 4min 58s, sys: 42.4 s, total: 2h 5min 41s
Wall time: 2h 5min 29s


,repo_name,path,function_name,function_code
0,trangvu/ape-npi,run-tests.py,failure,def failure(message):\n print('{}failure: {...
1,trangvu/ape-npi,run-tests.py,success,def success(message):\n print('{}success: {...
2,trangvu/ape-npi,run-tests.py,get_best_score,def get_best_score(log_file):\n scores = []...
3,trangvu/ape-npi,run-tests.py,run,"def run(dir_, score=None):\n config_file = ..."
4,trangvu/ape-npi,scripts/apply_bpe.py,BPE,"class BPE(object):\n\n def encode(self, ori..."
...,...,...,...,...
12537187,blessengeorge/compare_gan,compare_gan/src/multi_gan/visualize_gan.py,GetMultiGANGeneratorsOp,"def GetMultiGANGeneratorsOp(graph, gan_type, a..."
12537188,blessengeorge/compare_gan,compare_gan/src/multi_gan/visualize_gan.py,EvalCheckpoint,"def EvalCheckpoint(checkpoint_path, task_workd..."
12537189,blessengeorge/compare_gan,compare_gan/src/multi_gan/visualize_gan.py,GetModelDir,def GetModelDir(options):\n 'Returns the mo...
12537190,blessengeorge/compare_gan,compare_gan/src/multi_gan/visualize_gan.py,EvalTask,"def EvalTask(options, task_workdir, out_dir):\..."


In [37]:
python_functions_df = pd.read_csv("output/python_functions.csv")

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [37]:
python_functions_df.shape

NameError: name 'python_functions_df' is not defined

In [38]:
import matplotlib.pyplot as plt

plt.style.use("dark_background")

In [39]:
python_functions_df["repo_name"].value_counts()  # .plot.hist()
# plt.yscale("log")

NameError: name 'python_functions_df' is not defined

In [40]:
repo_function_counts = python_functions_df["repo_name"].value_counts()  # .plot.hist()
small_repos = repo_function_counts[repo_function_counts < 50]

NameError: name 'python_functions_df' is not defined

In [41]:
repo_sample = python_functions_df["repo_name"].sample(10)

NameError: name 'python_functions_df' is not defined

In [42]:
python_files_df[python_files_df["repo_name"].isin(repo_sample)].shape

NameError: name 'repo_sample' is not defined

In [43]:
small_repo_functions_df = python_functions_df[
    python_functions_df["repo_name"].isin(small_repos.index)
]

NameError: name 'python_functions_df' is not defined

In [44]:
paperswithcode_with_imports_df = pd.read_csv("output/papers_with_imports.csv")

<ipython-input-44-dbd5717aacfb>:1: DtypeWarning: Columns (2,15) have mixed types. Specify dtype option on import or set low_memory=False.
  paperswithcode_with_imports_df = pd.read_csv("output/papers_with_imports.csv")


In [45]:
paperswithcode_with_imports_df["tasks"]

0                                  ['Dictionary Learning']
1                                    ['Contour Detection']
2        ['Named Entity Recognition', 'Sentiment Analys...
3        ['Named Entity Recognition', 'Sentiment Analys...
4        ['Heart Rate Variability', 'Image-to-Image Tra...
                               ...                        
36204                                   ['Style Transfer']
36205                                   ['Style Transfer']
36206                                   ['Style Transfer']
36207                                   ['Style Transfer']
36208    ['Denoising', 'Domain Adaptation', 'Representa...
Name: tasks, Length: 36209, dtype: object

In [46]:
functions_df = pd.read_csv("output/python_functions.csv", index_col="Unnamed: 0")

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
example_functions_df = functions_df[functions_df["repo_name"] == "trangvu/ape-npi"]

In [ ]:
from mlutil.feature_extraction import embeddings_torch

In [33]:
embedding_model = embeddings_torch.TransformerVectorizer(
    model_type="microsoft/codebert-base", batch_size=256
)

OSError: /home/kuba/.local/lib/python3.8/site-packages/torch_scatter/_scatter_cuda.so: undefined symbol: _ZNK2at6Tensor6deviceEv

In [ ]:
code_embeddings = embedding_model.transform(list(example_functions_df["function_code"]))

In [ ]:
code_embeddings.shape

In [ ]:
from sklearn import cluster, metrics

In [ ]:
class PrincipalComponentRemover:
    def __init__(self, pca):
        self.pca = pca

    def transform(self, X):
        self.pca.fit(X)
        transformed_embeddings = self.pca.transform(X)
        transformed_embeddings[:, 0] = 0
        return self.pca.inverse_transform(transformed_embeddings)

In [ ]:
class CentroidSummarizer:
    def __init__(self, embedder, summary_size=10, pc_remover=None):
        self.embedder = embedder
        self.clusterer = cluster.KMeans(summary_size, n_jobs=-1)
        self.pc_remover = pc_remover

    def fit_pc_remover(self, texts):
        text_embeddings = self.embedder.transform(texts)
        pca = decomposition.PCA()
        pca.fit(text_embeddings)
        self.pc_remover = PrincipalComponentRemover(pca)

    def summarize(self, texts, **kwargs):
        text_embeddings = self.embedder.transform(texts, **kwargs)
        if self.pc_remover:
            text_embeddings = self.pc_remover.transform(text_embeddings)
        self.clusterer.fit(text_embeddings)
        centroids = self.clusterer.cluster_centers_
        centroid_distances = metrics.pairwise.euclidean_distances(
            centroids, text_embeddings
        )
        closest_idxs = centroid_distances.argsort(axis=1)
        return np.array(texts)[closest_idxs[:, 0]]

In [ ]:
repo_function_counts = functions_df["repo_name"].value_counts()

In [ ]:
repo_function_counts

In [ ]:
selected_repos = functions_df["repo_name"].unique()[:50]

In [ ]:
selected_repos = [repo for repo in selected_repos if repo_function_counts[repo] < 10000]

In [ ]:
len(selected_repos)

In [ ]:
from mlutil import summarization
from sklearn import decomposition

In [ ]:
function_sample = functions_df.sample(50000)
function_sample_embeddings = embedding_model.transform(
    list(function_sample["function_code"])
)

In [ ]:
pca = decomposition.PCA()

In [ ]:
pca.fit(function_sample_embeddings)

In [ ]:
pc_remover = PrincipalComponentRemover(pca)

In [ ]:
from sklearn import cluster, metrics, decomposition, mixture
import numpy as np


class PrincipalComponentRemover:
    def __init__(self, pca, zeroed_out_pcs=1):
        self.pca = pca
        self.zeroed_out_pcs = zeroed_out_pcs

    def transform(self, X):
        self.pca.fit(X)
        transformed_embeddings = self.pca.transform(X)
        transformed_embeddings[:, : self.zeroed_out_pcs] = 0
        return self.pca.inverse_transform(transformed_embeddings)


class CentroidSummarizer:
    def __init__(
        self, embedder, n_clusters=10, pc_remover=None, clusterer_cls=cluster.KMeans
    ):
        self.embedder = embedder
        self.clusterer = clusterer_cls(n_clusters)
        self.pc_remover = pc_remover

    def fit_clusterer(self, texts=None, text_embeddings=None, **kwargs):
        assert not texts is None or not text_embeddings is None
        if text_embeddings is None:
            text_embeddings = self.embedder.transform(texts)
        self.clusterer.fit(text_embeddings)

    def fit_pc_remover(self, texts=None, text_embeddings=None, zeroed_out_pcs=2):
        assert not texts is None or not text_embeddings is None
        if text_embeddings is None:
            text_embeddings = self.embedder.transform(texts)
        pca = decomposition.PCA()
        pca.fit(text_embeddings)
        self.pc_remover = PrincipalComponentRemover(pca, zeroed_out_pcs=zeroed_out_pcs)

    def summarize(self, texts, n_summaries=1, fit_clusterer=False, **kwargs):
        text_embeddings = self.embedder.transform(texts, **kwargs)
        if self.pc_remover:
            text_embeddings = self.pc_remover.transform(text_embeddings)
        if fit_clusterer:
            self.clusterer.fit(text_embeddings)
        centroids = self.clusterer.cluster_centers_
        centroid_distances = metrics.pairwise.cosine_distances(
            centroids, text_embeddings
        )
        closest_idxs_x, closest_idxs_y = np.unravel_index(
            centroid_distances.argsort(axis=None), centroid_distances.shape
        )
        return np.array(texts)[closest_idxs_y[:n_summaries]]

In [ ]:
summarizer = mlutil.summarization.CentroidSummarizer(
    embedding_model, n_clusters=500
)  # pca.components_[0])

In [ ]:
# summarizer.fit_pc_remover(text_embeddings=function_sample_embeddings)

In [ ]:
%%time
summarizer.fit_clusterer(text_embeddings=function_sample_embeddings)

In [69]:
import shutup

shutup.please()
import tqdm

In [70]:
summaries = [
    summarizer.summarize(
        list(functions_df["function_code"][functions_df["repo_name"] == repo]),
        verbose=False,
    )
    for repo in tqdm.tqdm(selected_repos)
]

100%|██████████| 48/48 [01:17<00:00,  1.61s/it]


In [71]:
repo_typical_functions_df = pd.DataFrame(
    {"repo": selected_repos, "summary": summaries}
).explode("summary")

repo_typical_functions_df

In [ ]:
repo_typical_functions_df[50:]

In [ ]:
list(summaries)

In [ ]:
summarizer.clusterer.cluster_centers_

In [ ]:
functions_with_tasks_df["tasks"] = functions_with_tasks_df["tasks"].apply(
    ast.literal_eval
)

In [ ]:
exploded_functions_with_tasks_df = functions_with_tasks_df.explode(
    column="tasks"
)  # .apply(ast.literal_eval).explode()

In [ ]:
exploded_functions_with_tasks_df

In [ ]:
import haystack

In [ ]:
paperswithcode_with_imports_df = pd.read_csv("output/papers_with_imports.csv")

In [ ]:
from sentence_transformers import SentenceTransformer

sentence_embedder = SentenceTransformer("microsoft/codebert-base")

In [ ]:
sentence_embedder = sentence_embedder.to("cuda")

In [ ]:
(exploded_functions_with_tasks_df["function_code"]).str.split().apply(len)

In [ ]:
n_samples = 1000

# Sentence transformers

In [ ]:
code_embeddings = sentence_embedder.encode(
    list(exploded_functions_with_tasks_df["function_code"].iloc[:n_samples])
)

In [ ]:
query_embeddings = sentence_embedder.encode(
    exploded_functions_with_tasks_df["tasks"].values
)

In [ ]:
from sentence_transformers import util as sentence_transformer_util

In [ ]:
queries = set(exploded_functions_with_tasks_df["tasks"].iloc[:n_samples])

In [ ]:
queries

In [ ]:
query_embeddings = sentence_embedder.encode(
    exploded_functions_with_tasks_df["tasks"].values
)

In [ ]:
cos_scores = sentence_transformer_util.pytorch_cos_sim(
    query_embeddings, code_embeddings
)

In [ ]:
cos_scores.shape

In [ ]:
import torch
import seaborn as sns

In [ ]:
diag_scores = torch.diag(cos_scores)

# Similarity between task and code embeddings for all tasks and matching tasks

In [ ]:
sns.distplot(cos_scores[:n_samples].reshape(-1), label="all", hist_kws=dict(alpha=0.5))
sns.distplot(diag_scores, label="matching", hist_kws=dict(alpha=0.5))
plt.legend()

In [ ]:
function_similarities_added_df = exploded_functions_with_tasks_df.iloc[
    :n_samples
].copy()

In [ ]:
function_similarities_added_df["similarity"] = torch.diag(cos_scores).cpu().numpy()

In [ ]:
sns.displot(
    function_similarities_added_df.groupby("function_name").agg("max")["similarity"]
)

In [ ]:
exploded_functions_with_tasks_df

# Cluster functions by tasks, path, function name

In [ ]:
import itertools
import tqdm

from sklearn import metrics, cluster

In [ ]:
import fasttext

In [ ]:
%%time
fasttext_model = fasttext.load_model("output/python_files_fasttext_dim200.bin")

In [ ]:
from mlutil.feature_extraction import embeddings

In [ ]:
fasttext_embedder = embeddings.FastTextVectorizer(fasttext_model)

In [ ]:
fname_with_path = (
    functions_with_tasks_df["path"].str.replace("/", " ")
    + " "
    + functions_with_tasks_df["function_name"]
    + " : "
    + functions_with_tasks_df["tasks"].apply(" ".join)
)

In [ ]:
fname_path_embeddings = fasttext_embedder.transform(fname_with_path)

In [ ]:
fname_path_cos_sim = metrics.pairwise.cosine_similarity(
    fname_path_embeddings, fname_path_embeddings
)

In [ ]:
fname_with_path.iloc[(-fname_path_cos_sim[100]).argsort()[:20]].str.split(":")

In [ ]:
kmeans = cluster.KMeans(n_clusters=20)

In [ ]:
cluster_labels = kmeans.fit_predict(fname_path_embeddings)

In [ ]:
for i in range(kmeans.n_clusters):
    print(fname_with_path[cluster_labels == i][:10])

## MLM model 

In [ ]:
from transformers import RobertaTokenizer, RobertaForMaskedLM
import torch

from sklearn import model_selection
from transformers import Trainer, TrainingArguments
from transformers import LineByLineTextDataset

from transformers import DataCollatorForLanguageModeling
import tqdm
from transformers import RobertaForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
model_name = "microsoft/codebert-base-mlm"

tokenizer = RobertaTokenizer.from_pretrained(model_name,truncation = True, padding=True, max_length=100)
model = RobertaForMaskedLM.from_pretrained(
    model_name,
    output_attentions = False, 
    output_hidden_states = True
)
device = 'cuda' if torch.cuda.is_available() else 'cpu' # Tell pytorch to run this model on the GPU.
model = model.to(device)
model.train()

In [ ]:
# tokenizer.add_tokens(["<tasks>", "<end_tasks>"])

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15,  # I don't know if this is the only way to set up mask for mlm task....
)

In [ ]:
function_about = (
    "<tasks> "
    + functions_with_tasks_df["tasks"].apply(" ".join).str.lower()
    + " <end_tasks>"
)

In [ ]:
functions_with_tasks = function_about + " " + functions_with_tasks_df["function_code"]
functions_with_tasks = (
    functions_with_tasks.str.replace("\n", "<n>")
    .str.split()
    .apply(lambda words: " ".join(words[:200]))
)

In [ ]:
functions_with_tasks_train, functions_with_tasks_val = model_selection.train_test_split(
    functions_with_tasks, test_size=10000
)

In [ ]:
functions_with_tasks_train.to_csv(
    "tmp/functions_about_train.txt", index=False, header=False
)
functions_with_tasks_val.to_csv(
    "tmp/functions_about_val.txt", index=False, header=False
)

In [ ]:
!head -2 tmp/functions_about.txt

In [ ]:
train_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="tmp/functions_about_train.txt",  # I reorgnasized data to line by line form as the tutorial, which is stupid. but  I also tried TensorDataset, it got errors
    block_size=100,
)

In [ ]:
eval_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="tmp/functions_about_val.txt",
    block_size=100,
)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=32,
    save_steps=1000,
    save_total_limit=2,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

In [ ]:
trainer.train()

In [ ]:
model.save_pretrained("tmp/codebert")

In [ ]:
trainer.evaluate(eval_dataset)

In [ ]:
model.device

In [ ]:
cpu_model = model.to("cpu")

In [ ]:
from transformers import pipeline

fill_mask = pipeline("fill-mask", model=cpu_model, tokenizer=tokenizer)

In [ ]:
example = functions_with_tasks_train.iloc[5]

In [ ]:
example = example.replace("anomaly detection", "<mask> <mask>")
example

In [ ]:
def get_perplexity_score(scoring_model, tokenizer, example, maxlen=500):
    inputs = tokenizer(example, return_tensors="pt")
    inputs = inputs.to(scoring_model.device)
    inputs["input_ids"] = inputs["input_ids"][:, :maxlen]
    inputs["attention_mask"] = inputs["attention_mask"][:, :maxlen]
    scoring_model_out = scoring_model(**inputs, labels=inputs["input_ids"])
    return scoring_model_out[0].mean().item()

In [ ]:
scoring_model = RobertaForCausalLM.from_pretrained(
    "tmp/codebert",
    is_decoder=True,
    output_attentions=True,
    output_hidden_states=False,
)

In [ ]:
gptn_tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")

gptn_scoring_model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")

In [ ]:
gptn_scoring_model.cuda()

# Evaluating whether model confuses tasks

In [ ]:
functions_with_tasks_permuted_train = functions_with_tasks_train.copy()

In [ ]:
functions_with_tasks_permuted_train.iloc[0]

In [ ]:
permuted_tasks_train = functions_with_tasks_permuted_train.str.findall(
    r"(<tasks>.*<end_tasks>)"
).apply(lambda l: l[0])

In [ ]:
permuted_tasks_train = permuted_tasks_train.sample(
    len(permuted_tasks_train)
).reset_index(drop=True)

In [ ]:
permuted_tasks_train.index = functions_with_tasks_permuted_train.index

In [ ]:
functions_with_tasks_permuted_train.iloc[0]

In [ ]:
functions_with_tasks_permuted_train = (
    permuted_tasks_train
    + " "
    + functions_with_tasks_permuted_train.str.replace(
        r"(FUNCTION_IS_ABOUT.*END_FUNCTION_IS_ABOUT)", ""
    )
)

In [ ]:
scores_df = pd.DataFrame.from_records(
    [
        {
            "original": get_perplexity_score(
                gptn_scoring_model, gptn_tokenizer, original
            ),
            "permuted": get_perplexity_score(
                gptn_scoring_model, gptn_tokenizer, permuted
            ),
        }
        for (original, permuted) in tqdm.tqdm(
            zip(
                functions_with_tasks_train.sample(1000).values,
                functions_with_tasks_permuted_train.values,
            ),
            total=1000,
        )
    ]
)

In [ ]:
scores_df.describe()

In [ ]:
(scores_df["original"] < scores_df["permuted"]).mean()

In [ ]:
get_perplexity_score(
    scoring_model, tokenizer, functions_with_tasks_permuted_train.iloc[1]
)

In [ ]:
get_perplexity_score(scoring_model, tokenizer, functions_with_tasks_train.iloc[1])

In [ ]:
functions_with_tasks_train.iloc[1]

In [ ]:
functions_with_tasks_permuted_train.iloc[1]

In [ ]:
get_perplexity_score(scoring_model, tokenizer, functions_with_tasks_train.iloc[:10])

In [ ]:
input_txt = example
inputs = tokenizer(input_txt, return_tensors="pt")

In [ ]:
inputs.keys()

In [ ]:
outputs = model(**inputs)
predictions = outputs[0]

In [ ]:
def get_prediction(sent, maxlen=512):

    token_ids = tokenizer.encode(sent, return_tensors="pt")[:, :maxlen]
    masked_position = (token_ids.squeeze() == tokenizer.mask_token_id).nonzero()
    masked_pos = [mask.item() for mask in masked_position]

    with torch.no_grad():
        output = model(token_ids)

    last_hidden_state = output[0].squeeze()

    list_of_list = []
    for index, mask_index in enumerate(masked_pos):
        mask_hidden_state = last_hidden_state[mask_index]
        idx = torch.topk(mask_hidden_state, k=5, dim=0)[1]
        words = [tokenizer.decode(i.item()).strip() for i in idx]
        list_of_list.append(words)
        print("Mask ", index + 1, "Guesses : ", words)

    best_guess = ""
    for j in list_of_list:
        best_guess = best_guess + " " + j[0]

    return best_guess.strip()

In [ ]:
get_prediction(example.replace("clustering", "<mask>"))

In [ ]:
torch.cuda.empty_cache()

In [ ]:
from transformers import pipeline

generator = pipeline("text-generation", model="EleutherAI/gpt-neo-2.7B", device=0)

In [ ]:
t = """
C:  def f1(x, y): return x + y
Q: what does this function do
A: addition
###
C:  def f2(x, y): return x * y
Q: what does this function do
A: """

In [ ]:
print(generator(t, do_sample=True, min_length=1)[0]["generated_text"])

In [ ]:
function_codes = (
    functions_with_tasks_df["function_code"]
    .str.split()
    .apply(lambda words: " ".join(words[:100]))
)

In [ ]:
function_tasks = functions_with_tasks_df["tasks"]

In [ ]:
functions_with_appended_tasks = (
    function_codes + "\n tasks: " + function_tasks.apply(", ".join) + "\n"
)

In [ ]:
functions_with_appended_tasks[:3]

In [ ]:
whole_example = " ".join(functions_with_appended_tasks.sample(5).values)

In [ ]:
print(whole_example)

In [ ]:
tasks = whole_example.split("tasks")[-1][1:-1]
cut_length = len(whole_example.split("tasks")[-1]) - 1

In [ ]:
tasks

In [ ]:
whole_example = whole_example[:-cut_length].rstrip()

In [ ]:
whole_example

In [ ]:
whole_example

In [ ]:
generator.model.device

In [ ]:
whole_example

In [ ]:
%%time
generator(whole_example, do_sample=True, min_length=3, max_length=1100)

In [ ]:
get_prediction(t)

In [ ]:
example = functions_with_tasks_train.iloc[0]
example = example.replace("atari games", "reinforcement learning")
example

In [ ]:
get_perplexity_score(scoring_model, tokenizer, example)

In [ ]:
model = model.save_pretrained("tmp/codebert")

In [ ]:
inputs["input_ids"]

In [ ]:
example

In [ ]:
sorted_preds, sorted_idx = predictions[0].sort(dim=-1, descending=True)
for k in range(2):
    predicted_index = [sorted_idx[i, k].item() for i in range(0, 24)]
    predicted_token = [
        tokenizer.convert_ids_to_tokens([predicted_index[x]])[0] for x in range(1, 24)
    ]
    print(predicted_token)
    words = [tokenizer.decode(t) for t in predicted_token]
    print("".join(predicted_token).replace("Ġ", " "))

In [ ]:
predicted_token

In [ ]:
pairwise_similarities = pd.DataFrame(
    data=None,
    index=minhash_amenable_fnames_with_path.values,
    columns=minhash_amenable_fnames_with_path.values,
)
for (s1, s2) in tqdm.tqdm(
    itertools.product(
        minhash_amenable_fnames_with_path.values,
        minhash_amenable_fnames_with_path.values,
    ),
    total=n ** 2,
):
    pairwise_similarities.loc[s1, s2] = shingles.text_similarity(s1, s2)

In [ ]:
pairwise_similarities

# Bitext mining approach